## Libraries and file imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import timedelta
from google.colab import drive

# Mount google drive
drive.mount('/content/drive')
base_path = '/content/drive/MyDrive/a/'
file_path = os.path.join(base_path, 'ukdale.h5')

Mounted at /content/drive


## Data loading and keys for H5 file

In UK-DALE H5 file:
*   meter1 = Mains (Whole House)
*   meter2, meter3, etc. = Individual Appliances (Check metadata for mapping)

In [2]:
# List all keys in the H5 file
def get_hdf_keys(path):
    with pd.HDFStore(path, mode='r') as store:
        keys = store.keys()
    return keys

all_keys = get_hdf_keys(file_path)
print(f"# datasets: {len(all_keys)}")

# datasets: 111


In [3]:
# Test sample for house 1
house_1_keys = [k for k in all_keys if 'building1' in k]
print("House 1 first ten meters:")
for k in house_1_keys[:10]:
    print(k)

House 1 first ten meters:
/building1/elec/meter1
/building1/elec/meter10
/building1/elec/meter11
/building1/elec/meter12
/building1/elec/meter13
/building1/elec/meter14
/building1/elec/meter15
/building1/elec/meter16
/building1/elec/meter17
/building1/elec/meter18


In [4]:
# Loads a specific meter, handles timestamps, and resamples data.
# We need to resample to 1-hr since dataset has 6-sec increments
# Meter 1 often has [Active Power, Voltage] so we only want the first Active Power.
def load_and_clean_meter(file_path, key, resample_rule='1h'):
    df = pd.read_hdf(file_path, key=key)
    if isinstance(df.index, pd.RangeIndex) or df.index[0] == 0:
        df.index = df.iloc[:, 0] # time
        df = df.iloc[:, [1]] # power

    df.index = pd.to_datetime(df.index, unit='ns', utc=True)

    if df.shape[1] > 1:
        df = df.iloc[:, [0]]

    # resampling
    df_resampled = df.resample(resample_rule).mean()
    df_resampled = df_resampled.ffill(limit=10).fillna(0)
    df_resampled.columns = ['power_watts']

    return df_resampled

mains_df = load_and_clean_meter(file_path, '/building1/elec/meter1/table', resample_rule='1h')

print(f"Data head:\n{mains_df.head()}")
print(f"\nData summary:\n{mains_df.describe()}")

Data head:
                           power_watts
index                                 
2012-11-09 22:00:00+00:00   474.804413
2012-11-09 23:00:00+00:00   442.738037
2012-11-10 00:00:00+00:00   608.406860
2012-11-10 01:00:00+00:00   826.403381
2012-11-10 02:00:00+00:00   173.122131

Data summary:
        power_watts
count  39092.000000
mean     371.787781
std      269.170288
min        0.000000
25%      201.590183
50%      273.632462
75%      461.896309
max     3472.787354


In [5]:
# trim down the dataset from three houses to two based on suitability for modeling due the sheer size of the dataset(s)
candidates = [1, 2, 5]

house_stats = []
final_dataset = {}

with pd.HDFStore(file_path, mode='r') as store:
    all_keys = store.keys()

    for house_id in candidates:
        base_key = f'/building{house_id}/elec/'
        mains_key = f'{base_key}meter1/table'

        house_meters = [k for k in all_keys if base_key in k]

        df_mains = pd.read_hdf(file_path, key=mains_key)

        if isinstance(df_mains.index, pd.RangeIndex) or df_mains.index[0] == 0:
            df_mains.index = df_mains.iloc[:, 0]
            df_mains = df_mains.iloc[:, [1]]

        df_mains.index = pd.to_datetime(df_mains.index, unit='ns', utc=True)

        # Ensure we only have one column (active power)
        if df_mains.shape[1] > 1:
              df_mains = df_mains.iloc[:, [0]]

        df_mains.columns = ['power_watts']

        start_date = df_mains.index.min()
        end_date = df_mains.index.max()
        duration_days = (end_date - start_date).days
        appliance_count = len(house_meters) - 1

        stats = {
            'House': f'House {house_id}',
            'Duration (Days)': duration_days,
            'Appliance Count': appliance_count,
            'Start Date': start_date.strftime('%Y-%m-%d'),
            'End Date': end_date.strftime('%Y-%m-%d')
        }
        house_stats.append(stats)

        if house_id in [1, 2]:
          # Resample to Hourly to save resources
          final_dataset[f'House_{house_id}'] = df_mains.resample('1h').mean().fillna(0)

stats_df = pd.DataFrame(house_stats).set_index('House')
print("\nHouse stats:")
print(stats_df[['Duration (Days)', 'Appliance Count', 'Start Date', 'End Date']])

# 3. Verify Final Dataset
print("\nFinal dataset:")
print(f"Houses kept: {list(final_dataset.keys())}")


House stats:
         Duration (Days)  Appliance Count  Start Date    End Date
House                                                            
House 1             1628               53  2012-11-09  2017-04-26
House 2              234               19  2013-02-17  2013-10-10
House 5              137               25  2014-06-29  2014-11-13

Final dataset:
Houses kept: ['House_1', 'House_2']


# Fill missing gaps and provide interpolation in prep for LSTM modeling

In [6]:
MAX_POWER_WATTS = 20000  # 20kW limit (typical UK main fuse is ~24kW)
INTERPOLATION_LIMIT = 24 # Hours to interpolate (fill gaps up to 1 day)

cleaned_dataset = {}

start_dates = []
end_dates = []

for house_id, df in final_dataset.items():
    start_dates.append(df.index.min())
    end_dates.append(df.index.max())

# Align on single start/end date from houses
common_start = max(start_dates)
common_end = min(end_dates)

for house_id, df in final_dataset.items():
    df_clean = df.copy()

    original_len = len(df_clean)
    df_clean = df_clean.loc[common_start:common_end]

    # Mains power is rarely exactly 0 so treat 0 as missing to interpolate
    zeros_count = (df_clean['power_watts'] == 0).sum()
    df_clean['power_watts'] = df_clean['power_watts'].replace(0, pd.NA)
    df_clean['power_watts'] = df_clean['power_watts'].interpolate(method='linear', limit=INTERPOLATION_LIMIT)
    df_clean['power_watts'] = df_clean['power_watts'].fillna(0)

    # Clip zeros and outliers
    df_clean['power_watts'] = df_clean['power_watts'].clip(lower=0, upper=MAX_POWER_WATTS)

    cleaned_dataset[house_id] = df_clean

    # Cleaning stats
    print(f"Stats for {house_id}:")
    print(f"  - Trimmed rows: {original_len} -> {len(df_clean)}")
    print(f"  - Interpolated {zeros_count} zero-readings")
    print(f"  - Min Power: {df_clean['power_watts'].min():.2f} W")
    print(f"  - Max Power: {df_clean['power_watts'].max():.2f} W\n")

final_dataset = cleaned_dataset

Stats for House_1:
  - Trimmed rows: 39092 -> 5630
  - Interpolated 31 zero-readings
  - Min Power: 0.00 W
  - Max Power: 2371.42 W

Stats for House_2:
  - Trimmed rows: 5630 -> 5630
  - Interpolated 826 zero-readings
  - Min Power: 0.00 W
  - Max Power: 2130.58 W



/tmp/ipython-input-431865693.py:26: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_clean['power_watts'] = df_clean['power_watts'].interpolate(method='linear', limit=INTERPOLATION_LIMIT)
/tmp/ipython-input-431865693.py:27: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_clean['power_watts'] = df_clean['power_watts'].fillna(0)
/tmp/ipython-input-431865693.py:26: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_clean['power_watts'] = df_clean['power_watts'].interpolate(method='linear', limit=INTERPOLATION_LIMIT)
/tmp/ipython-inp

## Export data back to Google Drive as csv for future analysis

In [8]:
for house_id, df in final_dataset.items():
    csv_name = f"{house_id}_cleaned.csv"
    csv_path = os.path.join(base_path, csv_name)
    df.to_csv(csv_path)
    print(f"saved: {csv_name}")

saved: House_1_cleaned.csv
saved: House_2_cleaned.csv
